### pip install

In [1]:
!pip install torch

In [2]:
!pip install torchmetrics

In [3]:
!pip install pandas

In [4]:
!pip install torchvision==0.9.0 torchaudio==0.8.0 pytorch-lightning==1.2.2

  Using cached torchvision-0.9.0-cp37-cp37m-manylinux1_x86_64.whl (17.3 MB)
  Using cached pytorch_lightning-1.2.2-py3-none-any.whl (816 kB)
  Using cached torch-1.8.0-cp37-cp37m-manylinux1_x86_64.whl (735.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.5.10
    Uninstalling pytorch-lightning-1.5.10:
      Successfully uninstalled pytorch-lightning-1.5.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.5.3 requires spacy<4, which is not installed.
torchtext 0.12.0 requires torch==

In [5]:
import torch
print(torch.__version__)

1.8.0


### import library

In [6]:
import os
import torch
import glob
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import math
import torchmetrics
import logging
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models
import torchvision.models as models
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pytorch_lightning import LightningModule

import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.utils.data import random_split
from typing import Dict, List, Optional, Sequence, Tuple, Type, Union
from torch import Tensor
from pytorch_lightning import LightningDataModule
from torchvision import transforms as T
# from warnings import warn
# from kornia import augmentation, geometry, image_to_tensor

ImportError: cannot import name 'functional_datapipe' from 'torch.utils.data' (/home/tungnguyendinh/interns/env/lib/python3.7/site-packages/torch/utils/data/__init__.py)

In [ ]:
cd /home/tungnguyendinh/interns/users/hoa_hoangthi/data/pascal_2007

### data exploration

In [ ]:
data_train = pd.read_csv('./train.csv')
print(data_train)
data_train.head()

In [ ]:
import itertools

mul_labels = list(itertools.chain(*[lbs.split(" ") for lbs in data_train['labels']]))

In [ ]:
ax = sns.countplot(y=sorted(mul_labels), orient='v')
ax.grid()

In [ ]:
data_test = pd.read_csv('./test.csv')
print(data_test)
data_test.head()

In [ ]:
labels_unique = set(mul_labels)
print(f"unique labels: {labels_unique}")
data_train['labels_sorted'] = [" ".join(sorted(lbs.split(" "))) for lbs in data_train['labels']]

labels_combine = {}
for comb in data_train['labels_sorted']:
    labels_combine[comb] = labels_combine.get(comb, 0) + 1

show_counts = '\n'.join(sorted(f'\t{k}: {v}' for k, v in labels_combine.items()))
print(f"unique combinations: \n" + show_counts)
print(f"total: {sum(labels_combine.values())}")

In [ ]:
labels = ['dog', 'tvmonitor', 'person', 'bird', 'chair', 'car', 'sofa', 'sheep', 'bottle', 'bicycle', 'motorbike', 'bus', 'boat', 'pottedplant', 'horse', 'cat', 'aeroplane', 'cow', 'train', 'diningtable']
mul_labels = labels

In [ ]:
data_train = pd.read_csv('./train.csv')
label_train = []
import cv2
for i in range(len(data_train)):
  vitri = 0
  lb = np.zeros(20)
  for j in mul_labels:
    labels = data_train['labels'][i].split(' ')
    for k in labels:
      if k == j:
        lb[vitri]=1
    vitri+=1
  label_train.append(lb)

data_train['label_train'] = label_train
print(data_train['label_train'][2])
data_train

### data preparation

In [ ]:
DATASET_IMAGE_MEAN = (0.48690377, 0.62658835, 0.4078062)
DATASET_IMAGE_STD = (0.18142496, 0.15883319, 0.19026241)

In [ ]:
class Pascal2007Dataset(Dataset):

    def __init__(
        self,
        df_data: Union[str, pd.DataFrame] = 'train.csv',
        path_img_dir: str = './train',
        transforms=None,
        mode: str = 'train',
        split: float = 0.8,
        uq_labels: Tuple[str] = None,
        random_state=42,
    ):
        self.path_img_dir = path_img_dir
        self.transforms = transforms
        self.mode = mode

        # set or load the config table
        if isinstance(df_data, pd.DataFrame):
            self.data = df_data
        elif isinstance(df_data, str):
            assert os.path.isfile(df_data), f"missing file: {df_data}"
            self.data = pd.read_csv(df_data)
        else:
            raise ValueError(f'unrecognised input for DataFrame/CSV: {df_data}')

        # take over existing table or load from file
        if uq_labels:
            self.labels_unique = tuple(uq_labels)
        else:
            labels_all = list(itertools.chain(*[lbs.split(" ") for lbs in self.raw_labels]))
            self.labels_unique = tuple(sorted(set(labels_all)))
        self.labels_lut = {lb: i for i, lb in enumerate(self.labels_unique)}
        self.num_classes = len(self.labels_unique)

        # shuffle data
        self.data = self.data.sample(frac=1, random_state=random_state).reset_index(drop=True)

        # split dataset
        assert 0.0 <= split <= 1.0, f"split {split} is out of range"
        frac = int(split * len(self.data))
        self.data = self.data[:frac] if mode == 'train' else self.data[frac:]
        self.img_names = list(self.data['fname'])
        self.labels = self._prepare_labels()
        # compute importance order
        self.label_importance_index = []

    @property
    def raw_labels(self):
        return list(self.data['labels'])

    def _prepare_labels(self) -> list:
        return [torch.tensor(self.to_binary_encoding(lb)) if lb else None for lb in self.raw_labels]

    @property
    def label_histogram(self) -> Tensor:
        lb_stack = torch.tensor(list(map(tuple, self.labels)))
        return torch.sum(lb_stack, dim=0)

    def to_binary_encoding(self, labels: str) -> tuple:
        # processed with encoding
        one_hot = [0] * len(self.labels_unique)
        for lb in labels.split(" "):
            one_hot[self.labels_lut[lb]] = 1
        return tuple(one_hot)

    def __getitem__(self, idx: int) -> tuple:
        img_name = self.img_names[idx]
        img_path = os.path.join(self.path_img_dir, img_name)
        assert os.path.isfile(img_path)
        label = self.labels[idx]
        img = plt.imread(img_path)

        # augmentation
        if self.transforms:
            img = self.transforms(Image.fromarray(img))
        # in case of predictions, return image name as label
        label = label if label is not None else img_name
        return img, label

    def __len__(self) -> int:
        return len(self.data)

    def get_sample_pseudo_label(self, idx: int):
        if not self.label_importance_index:
            idx_nb = list(enumerate(self.label_histogram))
            idx_nb = sorted(idx_nb, key=lambda x: x[1])
            self.label_importance_index = [i[0] for i in idx_nb]
        binary = self.labels[idx]
        # take the less occurred label, not the tuple combination as combination does not matter too much
        for i in self.label_importance_index:
            if binary[i]:
                return i
        # this is a failer...
        return tuple(binary.numpy())

    def get_sample_pseudo_labels(self, *_):
        return [self.get_sample_pseudo_label(i) for i in range(len(self))]

In [ ]:
class Pascal2007SimpleDataset(Pascal2007Dataset):
    
    def _translate_labels(self, lb):
        if lb is None:
            return None
        lb = self.labels_lut['complex'] if torch.sum(lb) > 1 else torch.argmax(lb)
        return int(lb)

    def _prepare_labels(self) -> list:
        labels = super()._prepare_labels()
        return list(map(self._translate_labels, labels))

    @property
    def label_histogram(self) -> Tensor:
        if not isinstance(self.labels, Tensor):
            self.labels = torch.tensor(self.labels)
        return torch.bincount(self.labels)

    def get_sample_pseudo_label(self, idx: int):
        return self.labels[idx]

In [ ]:
class Pascal2007DM(LightningDataModule):

    def __init__(
        self,
        path_csv: str = 'train.csv',
        base_path: str = '.',
        batch_size: int = 32,
        num_workers: int = None,
        simple: bool = False,
        train_transforms=None,
        valid_transforms=None,
        split: float = 0.8,
        balancing: bool = False,
    ):
        super().__init__()
        # path configurations
        assert os.path.isdir(base_path), f"missing folder: {base_path}"
        self.train_dir = os.path.join(base_path, 'train')
        self.test_dir = os.path.join(base_path, 'test')

        if not os.path.isfile(path_csv):
            path_csv = os.path.join(base_path, path_csv)
        assert os.path.isfile(path_csv), f"missing table: {path_csv}"
        self.path_csv = path_csv

        # other configs
        self.batch_size = batch_size
        self.split = split
        self.num_workers = num_workers if num_workers is not None else os.cpu_count()
        self.labels_unique: Sequence = ...
        self.lut_label: Dict = ...
        self.label_histogram: Tensor = ...
        self.balancing = balancing

        # need to be filled in setup()
        self.train_dataset = None
        self.valid_dataset = None
        self.test_table = []
        self.test_dataset = None
        self.train_transforms = train_transforms 
        self.valid_transforms = valid_transforms 
        self.dataset_cls: Type = Pascal2007SimpleDataset if simple else Pascal2007Dataset

    def prepare_data(self):
        pass

    @property
    def num_classes(self) -> int:
        assert self.train_dataset and self.valid_dataset
        return max(self.train_dataset.num_classes, self.valid_dataset.num_classes)

    @staticmethod
    def binary_mapping(
        encoding: Tensor,
        lut_label: Dict[int, str],
        thr: float = 0.5,
        label_required: bool = True,
    ) -> Union[str, List[str]]:
        """Convert Model outputs to string labels
        Args:
            encoding: one-hot encoding
            lut_label: look-up-table with labels
            thr: threshold for label binarization
            label_required: if it is required to return any label and no label is above `thr`, use argmax
        """
        assert lut_label
        # on case it is not one hot encoding but single label
        if encoding.nelement() == 1:
            return lut_label[encoding[0]]
        labels = [lut_label[i] for i, s in enumerate(encoding) if s >= thr]
        # in case no reached threshold then take max
        if not labels and label_required:
            idx = torch.argmax(encoding).item()
            labels = [lut_label[idx]]
        return sorted(labels)

    def binary_encoding_to_labels(
        self,
        encoding: Tensor,
        thr: float = 0.5,
        label_required: bool = True,
    ) -> Union[str, List[str]]:
        """Convert Model outputs to string labels
        Args:
            encoding: one-hot encoding
            thr: threshold for label binarization
            label_required: if it is required to return any label and no label is above `thr`, use argmax
        """
        return self.binary_mapping(encoding, self.lut_label, thr=thr, label_required=label_required)

    def setup(self, *_, **__) -> None:
        """Prepare datasets"""
        assert os.path.isdir(self.train_dir), f"missing folder: {self.train_dir}"
        ds = self.dataset_cls(self.path_csv, self.train_dir, mode = 'train',split=1.0)
        self.labels_unique = ds.labels_unique
        self.label_histogram = ds.label_histogram
        self.lut_label = dict(enumerate(self.labels_unique))

        ds_defaults = dict(
            df_data=self.path_csv,
            path_img_dir=self.train_dir,
            split=self.split,
            uq_labels=self.labels_unique,
        )
        self.train_dataset = self.dataset_cls(**ds_defaults, mode='train', transforms=self.train_transforms)
        logging.info(f"training dataset: {len(self.train_dataset)}")
        self.valid_dataset = self.dataset_cls(**ds_defaults, mode='valid', transforms=self.valid_transforms)
        logging.info(f"validation dataset: {len(self.valid_dataset)}")

        if not os.path.isdir(self.test_dir):
            return
        ls_images = glob.glob(os.path.join(self.test_dir, '*.*'))
        ls_images = [os.path.basename(p) for p in ls_images if os.path.splitext(p)[-1] in IMAGE_EXTENSIONS]
        self.test_table = [dict(image=n, labels='') for n in ls_images]
        self.test_dataset = self.dataset_cls(
            df_data=pd.DataFrame(self.test_table),
            path_img_dir=self.test_dir,
            split=0,
            uq_labels=self.labels_unique,
            mode='test',
            transforms=self.valid_transforms
        )
        logging.info(f"test dataset: {len(self.test_dataset)}")

    def _dataloader_extra_args(self, dataset: Pascal2007Dataset) -> dict:
        dl_kwargs = dict(shuffle=True)
        # if you ask and you have it
        if self.balancing and ImbalancedDatasetSampler:
            dl_kwargs = dict(
                shuffle=False,
                sampler=ImbalancedDatasetSampler(
                    dataset=dataset,
                    callback_get_label=self.dataset_cls.get_sample_pseudo_labels,
                )
            )
        elif self.balancing:
            warn('You have asked for `ImbalancedDatasetSampler` but you do not have it installed.')
        return dl_kwargs

    def train_dataloader(self) -> DataLoader:
        dl_kwargs = self._dataloader_extra_args(self.train_dataset)
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            **dl_kwargs,
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.valid_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
        )

    def test_dataloader(self) -> Optional[DataLoader]:
        if self.test_dataset:
            return DataLoader(
                self.test_dataset,
                batch_size=self.batch_size,
                num_workers=0,
                shuffle=False,
            )
        logging.warning('no testing images found')

In [ ]:
dataset = Pascal2007Dataset()

In [ ]:
fig = plt.figure(figsize=(20, 9), facecolor='white')
for i in range(9):
    img, lb = dataset[i]
    ax = fig.add_subplot(3, 3, i + 1, xticks=[], yticks=[])
    ax.imshow(img)
    ax.set_title(lb)

### model

In [ ]:
class LitResnet(nn.Module):

    def __init__(self, arch: str, pretrained: bool = True, num_classes: int = 6):
        super().__init__()
        self.arch = arch
        self.num_classes = num_classes
        self.model = torchvision.models.__dict__[arch](pretrained=pretrained)
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.model(x)


class LitPlantPathology(LightningModule):
    
    def __init__(
        self,
        model: Union[nn.Module, str] = "ResNet50",
        lr: float = 1e-4,
        augmentations: Optional[nn.Module] = None,
    ):
        super().__init__()
        if isinstance(model, str):
            model = LitResnet(arch=model)
        self.model = model
        self.arch = model.arch
        self.num_classes = model.num_classes
        self.train_accuracy = torchmetrics.Accuracy()
        self.train_precision = torchmetrics.Precision(**self._metrics_extra_args)
        self.train_f1_score = torchmetrics.F1(**self._metrics_extra_args)
        self.val_accuracy = torchmetrics.Accuracy()
        self.val_precision = torchmetrics.Precision(**self._metrics_extra_args)
        self.val_f1_score = torchmetrics.F1(**self._metrics_extra_args)
        self.learning_rate = lr
        self.aug = augmentations

    @property
    def _metrics_extra_args(self):
        return dict()

    def forward(self, x: Tensor) -> Tensor:
        return F.softmax(self.model(x))

    def compute_loss(self, y_hat: Tensor, y: Tensor):
        return F.cross_entropy(y_hat, y)

    def training_step(self, batch, batch_idx):
        x, y = batch
        if self.aug:
            x = self.aug(x)  # => batched augmentations
        y_hat = self(x)
        loss = self.compute_loss(y_hat, y)
        self.log("train_loss", loss, prog_bar=False)
        self.log("train_acc", self.train_accuracy(y_hat, y), prog_bar=False)
        self.log("train_prec", self.train_precision(y_hat, y), prog_bar=False)
        self.log("train_f1", self.train_f1_score(y_hat, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.compute_loss(y_hat, y)
        self.log("valid_loss", loss, prog_bar=False)
        self.log("valid_acc", self.val_accuracy(y_hat, y), prog_bar=True)
        self.log("valid_prec", self.val_precision(y_hat, y), prog_bar=True)
        self.log("valid_f1", self.val_f1_score(y_hat, y), prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, self.trainer.max_epochs, 0)
        return [optimizer], [scheduler]


class MultiPascal2007(LitPlantPathology):

    @property
    def _metrics_extra_args(self):
        return dict(num_classes=self.num_classes, average='weighted')

    def forward(self, x: Tensor) -> Tensor:
        return torch.sigmoid(self.model(x))

    def compute_loss(self, y_hat: Tensor, y: Tensor):
        return F.binary_cross_entropy_with_logits(y_hat, y.to(y_hat.dtype))

In [ ]:
IMAGE_EXTENSIONS = ('.png', '.jpg', '.jpeg')

In [ ]:
TORCHVISION_TRAIN_TRANSFORM = T.Compose([
    T.Resize(size=300, interpolation=Image.BILINEAR),
    T.RandomRotation(degrees=30),
    T.RandomPerspective(distortion_scale=0.4),
    T.RandomResizedCrop(size=224),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(DATASET_IMAGE_MEAN, DATASET_IMAGE_STD),  # custom
])
#: default validation augmentation
TORCHVISION_VALID_TRANSFORM = T.Compose([
    T.Resize(size=300, interpolation=Image.BILINEAR),
    T.CenterCrop(size=224),
    T.ToTensor(),
    T.Normalize(DATASET_IMAGE_MEAN, DATASET_IMAGE_STD),  # custom
])


class Resize(nn.Module):

    def __init__(self, size: int):
        super().__init__()
        self.size = size

    def forward(self, x):
        return geometry.resize(x[None], self.size)[0]


class LitPreprocess(nn.Module):
    """Applies the processing to the image in the worker before collate."""

    def __init__(self, img_size: int):
        super().__init__()
        self.preprocess = nn.Sequential(
            Resize((img_size, img_size)),  # use this better to see whole image
        )

    @torch.no_grad()
    def forward(self, x: Tensor) -> Tensor:
        x = image_to_tensor(np.array(x)).float() / 255.
        assert len(x.shape) == 3, x.shape
        out = self.preprocess(x)
        return out[0]


class LitAugmenter(nn.Module):
    """Applies random augmentation to a batch of images."""

    def __init__(self, viz: bool = False):
        super().__init__()
        self.viz = viz


    @torch.no_grad()
    def forward(self, x: Tensor) -> Tensor:
        assert len(x.shape) == 4, x.shape
        out = x
        out = self.augmentations(out)
        if self.viz:
            out = self.denorm(out)
        return out


KORNIA_TRAIN_TRANSFORM = LitPreprocess(512)
KORNIA_VALID_TRANSFORM = LitPreprocess(224)

In [ ]:
class PlantPathologyDM(LightningDataModule):

    def __init__(
        self,
        path_csv: str = 'train.csv',
        base_path: str = '.',
        batch_size: int = 32,
        num_workers: int = None,
        simple: bool = False,
        train_transforms=None,
        valid_transforms=None,
        split: float = 0.8,
        balancing: bool = False,
    ):
        super().__init__()
        # path configurations
        assert os.path.isdir(base_path), f"missing folder: {base_path}"
        self.train_dir = os.path.join(base_path, 'train')
        self.test_dir = os.path.join(base_path, 'test')

        if not os.path.isfile(path_csv):
            path_csv = os.path.join(base_path, path_csv)
        assert os.path.isfile(path_csv), f"missing table: {path_csv}"
        self.path_csv = path_csv

        # other configs
        self.batch_size = batch_size
        self.split = split
        self.num_workers = num_workers if num_workers is not None else os.cpu_count()
        self.labels_unique: Sequence = ...
        self.lut_label: Dict = ...
        self.label_histogram: Tensor = ...
        self.balancing = balancing

        # need to be filled in setup()
        self.train_dataset = None
        self.valid_dataset = None
        self.test_table = []
        self.test_dataset = None
        self.train_transforms = train_transforms or KORNIA_TRAIN_TRANSFORM
        self.valid_transforms = valid_transforms or KORNIA_VALID_TRANSFORM
        self.dataset_cls: Type = Pascal2007SimpleDataset if simple else Pascal2007Dataset

In [ ]:
dm = Pascal2007DM(
    simple = False,
    batch_size=98,
    train_transforms=TORCHVISION_TRAIN_TRANSFORM,
    valid_transforms=TORCHVISION_VALID_TRANSFORM,
    split=0.9,
    balancing=True,
)

In [ ]:
dm.setup()

In [ ]:
net = LitResnet(arch='resnext101_32x8d', num_classes=dm.num_classes)

In [ ]:
model = MultiPascal2007(model = net, lr = 6e-4)

In [ ]:
try:
    from torchsampler import ImbalancedDatasetSampler
except ImportError:
    ImbalancedDatasetSampler = None

In [ ]:
import pytorch_lightning as pl
print(pl.__version__)

logger = pl.loggers.CSVLogger(save_dir='logs/', name=model.arch)
swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=0.6)
ckpt = pl.callbacks.ModelCheckpoint(
    monitor='valid_f1',
    save_top_k=1,
    save_last=True,
    # save_weights_only=True,
    filename='checkpoint/{epoch:02d}-{valid_acc:.4f}-{valid_f1:.4f}',
    # verbose=False,
    mode='max',
)

# ==============================

trainer = pl.Trainer(
    # fast_dev_run=True,
    gpus=2,
    callbacks=[ckpt, swa, MyPrintingCallback()],
    logger=logger,
    max_epochs=1,
    precision=16,
    # overfit_batches=5,
    auto_lr_find=True,
    accumulate_grad_batches=4,
    val_check_interval=0.5,
    progress_bar_refresh_rate=1,
    weights_summary='top',
)

trainer.fit(model=model, datamodule=dm)

### note

In [ ]:
import gc
gc.collect()

In [ ]:
!nvidia-smi

### callback

In [ ]:
from pytorch_lightning.callbacks import Callback

class MyPrintingCallback(Callback):
    def on_inint_start(self, trainer):
        print("Bat dau khoi tao trainer")
    
    def on_init_end(self, trainer):
        print("trainer bat dau")
        
    def on_train_end(self, trainer, pl_module):
        print("lam gi do den khi traning ket thuc")
        
trainner = pl.Trainer(callbacks=[MyPrintingCallback()])
        

In [ ]:
# class Mycallback(pl.Callback):
#     def on_pretrain_routime_stat()
#     def on_pretrain_routime_end()
#     def on_train_start()
#     def on_train_epoch_start()
#     def on_tran_batch_start()
#     def optimizer_step()
#     def on_before_zero_grad()
#     def on_train_batch_end()
#     def on_validation_epoch_start()
#     def on_validatin_batch_start()
#     def on_validation_epoch_end()
#     def on_train_end()
#     def teardown()'